# Machine Learning

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
import os
from sklearn.linear_model import LogisticRegression

In [2]:
df_train=pd.read_csv("./../data/application_train.csv",encoding="utf-8",index_col=0)
df_test=pd.read_csv("./../data/application_test.csv",encoding="utf-8",index_col=0)
df = pd.concat([df_train,df_test])
df=df.dropna()
df

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100083,0.0,Cash loans,M,Y,Y,0,103500.0,573628.5,24435.0,463500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100145,0.0,Cash loans,F,Y,Y,1,202500.0,260725.5,16789.5,198000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100179,0.0,Cash loans,F,Y,N,0,202500.0,675000.0,53329.5,675000.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
100190,0.0,Cash loans,M,Y,N,0,162000.0,263686.5,24781.5,238500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100295,1.0,Cash loans,M,Y,N,1,225000.0,1019205.0,31032.0,774000.0,...,1,0,0,0,0.0,0.0,0.0,6.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456083,0.0,Cash loans,F,Y,Y,2,112500.0,361462.5,16051.5,274500.0,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0
456084,0.0,Cash loans,F,Y,Y,1,99000.0,675000.0,21906.0,675000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
456140,1.0,Cash loans,F,Y,Y,1,261000.0,711454.5,47673.0,643500.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0


In [3]:
total_corr = df.corr()
total_corr

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
TARGET,1.000000,-0.019993,-0.039762,-0.014634,-0.000765,-0.021950,-0.021225,0.041108,0.062950,0.013710,...,-0.006139,0.013976,-0.008259,-0.003892,0.012156,0.014616,0.015000,-0.004202,0.016465,0.033832
CNT_CHILDREN,-0.019993,1.000000,-0.023069,-0.015191,-0.020395,-0.021157,-0.025688,0.175747,0.014600,0.141624,...,-0.010280,-0.012527,-0.006602,-0.011857,-0.009919,-0.002712,-0.001043,-0.023803,-0.022028,-0.035499
AMT_INCOME_TOTAL,-0.039762,-0.023069,1.000000,0.348575,0.415807,0.354556,0.211133,-0.119206,-0.028260,0.005067,...,0.041473,0.010493,0.003233,0.031045,0.026559,-0.001641,0.024093,0.024042,0.006707,-0.007158
AMT_CREDIT,-0.014634,-0.015191,0.348575,1.000000,0.751587,0.987994,0.110804,-0.198152,-0.068760,-0.024419,...,0.061362,0.044737,0.073881,0.016894,0.007742,0.018995,0.012926,0.056718,0.003209,-0.057768
AMT_ANNUITY,-0.000765,-0.020395,0.415807,0.751587,1.000000,0.756896,0.142985,-0.151530,-0.058623,-0.010262,...,0.010720,0.027078,0.040405,0.078556,0.016238,0.014015,0.026397,0.025826,0.016595,-0.004702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AMT_REQ_CREDIT_BUREAU_DAY,0.014616,-0.002712,-0.001641,0.018995,0.014015,0.019636,-0.006398,0.018687,0.025302,0.004604,...,0.005988,-0.001495,-0.002006,-0.000945,0.214817,1.000000,0.215462,-0.002107,-0.014587,0.003018
AMT_REQ_CREDIT_BUREAU_WEEK,0.015000,-0.001043,0.024093,0.012926,0.026397,0.014701,-0.000075,0.003329,0.004644,0.003656,...,-0.007608,-0.004512,-0.006055,-0.002853,0.021885,0.215462,1.000000,-0.012199,-0.043595,0.024866
AMT_REQ_CREDIT_BUREAU_MON,-0.004202,-0.023803,0.024042,0.056718,0.025826,0.053417,0.081290,-0.036862,-0.014233,-0.016350,...,-0.009728,-0.003885,-0.004528,-0.005364,0.013652,-0.002107,-0.012199,1.000000,-0.035314,-0.010767
AMT_REQ_CREDIT_BUREAU_QRT,0.016465,-0.022028,0.006707,0.003209,0.016595,0.004426,-0.024234,-0.014376,0.025230,0.007577,...,-0.006256,-0.010430,-0.013997,-0.006595,0.000241,-0.014587,-0.043595,-0.035314,1.000000,0.088951


In [4]:
def get_coor(mat, corr = .75):
    clmn_lst = mat.columns
    exeption = [] 
    val = 0
    for clmn in mat:
        cmd = lambda idx, val: [print("{} -> {} : ".format(clmn, clmn_lst[idx]).ljust(80, ' ') + str(val)),  exeption.append(val)]
        [cmd(idx, val) for idx, val in enumerate(mat[clmn]) if val > corr and val != 1 and val not in exeption]

        

get_coor(total_corr, .99)




APARTMENTS_AVG -> APARTMENTS_MEDI :                                             0.9921889092102557
YEARS_BUILD_AVG -> YEARS_BUILD_MEDI :                                           0.9982194480086191
COMMONAREA_AVG -> COMMONAREA_MEDI :                                             0.9968669204774225
ELEVATORS_AVG -> ELEVATORS_MEDI :                                               0.9955720179330168
ENTRANCES_AVG -> ENTRANCES_MEDI :                                               0.9955845901184158
FLOORSMAX_AVG -> FLOORSMAX_MEDI :                                               0.9967160886728837
FLOORSMIN_AVG -> FLOORSMIN_MEDI :                                               0.9966117650318823
LIVINGAPARTMENTS_AVG -> LIVINGAPARTMENTS_MEDI :                                 0.9906545685652971
LIVINGAREA_AVG -> LIVINGAREA_MEDI :                                             0.9968557116101835
OBS_30_CNT_SOCIAL_CIRCLE -> OBS_60_CNT_SOCIAL_CIRCLE :                          0.9983833729587834


In [5]:
def get_clmn_corr(mat, clmn, corr = 0.1):
    serie = mat[clmn]
    clmn_lst = list(serie.keys())
    cmd = lambda idx, val: print("{} -> {} : ".format(clmn, clmn_lst[idx]).ljust(80, ' ') + str(val))
    [cmd(idx, val) for idx, val in enumerate(mat[clmn]) if abs(val) > corr and val != 1]
    
get_clmn_corr(total_corr, 'TARGET', 0.05)

TARGET -> DAYS_EMPLOYED :                                                       0.06295036668984247
TARGET -> REGION_RATING_CLIENT :                                                0.05361424825765006
TARGET -> REGION_RATING_CLIENT_W_CITY :                                         0.06121399994630334
TARGET -> EXT_SOURCE_1 :                                                        -0.12954752718226564
TARGET -> EXT_SOURCE_2 :                                                        -0.1309687926621415
TARGET -> EXT_SOURCE_3 :                                                        -0.1507200772848682
TARGET -> FLAG_DOCUMENT_3 :                                                     0.05426903240732253


In [6]:
colonnes=[colonne for colonne in df]
types_colonnes=list(map(lambda x : {x : list(set(df[x].map(lambda x : str(type(x)))))} , colonnes))

In [7]:
colonnes_str=list(filter(lambda x : x[list(x)[0]]==["<class 'str'>"], types_colonnes))

In [8]:
def vectorisation(col):
    voc=list(set(list(col)))
    val={voc[i]:i for i in range(len(voc))}
    return col.map(lambda x : val[x])



for i in [list(i)[0] for i in colonnes_str]:
    df[i]=vectorisation(df[i])

In [9]:
df

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100083,0.0,0,0,0,0,0,103500.0,573628.5,24435.0,463500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100145,0.0,0,1,0,0,1,202500.0,260725.5,16789.5,198000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100179,0.0,0,1,0,1,0,202500.0,675000.0,53329.5,675000.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
100190,0.0,0,0,0,1,0,162000.0,263686.5,24781.5,238500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100295,1.0,0,0,0,1,1,225000.0,1019205.0,31032.0,774000.0,...,1,0,0,0,0.0,0.0,0.0,6.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456083,0.0,0,1,0,0,2,112500.0,361462.5,16051.5,274500.0,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0
456084,0.0,0,1,0,0,1,99000.0,675000.0,21906.0,675000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
456140,1.0,0,1,0,0,1,261000.0,711454.5,47673.0,643500.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0


In [10]:
type(type(''))

type

In [11]:
set(df_train['FONDKAPREMONT_MODE'].fillna("").map(type))

{str}

In [12]:
df.corr()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
TARGET,1.000000,-0.045993,-0.033103,NaN,-0.003398,-0.019993,-0.039762,-0.014634,-0.000765,-0.021950,...,-0.006139,0.013976,-0.008259,-0.003892,0.012156,0.014616,0.015000,-0.004202,0.016465,0.033832
NAME_CONTRACT_TYPE,-0.045993,1.000000,-0.000923,NaN,-0.078037,0.020854,-0.030260,-0.248492,-0.271951,-0.212679,...,-0.020273,-0.008457,-0.011349,0.043487,-0.007084,-0.013953,-0.017467,-0.019016,-0.022528,-0.063929
CODE_GENDER,-0.033103,-0.000923,1.000000,NaN,-0.056212,0.020016,-0.104820,0.033866,-0.006283,0.033341,...,-0.000468,0.014921,0.018585,-0.014969,-0.013602,-0.004925,0.005709,-0.016955,-0.001697,-0.017062
FLAG_OWN_CAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FLAG_OWN_REALTY,-0.003398,-0.078037,-0.056212,NaN,1.000000,-0.029913,0.005579,0.041779,-0.007923,0.048085,...,0.096187,0.025383,0.040270,0.022632,-0.018160,0.013311,0.004906,0.009066,-0.030353,-0.068681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AMT_REQ_CREDIT_BUREAU_DAY,0.014616,-0.013953,-0.004925,NaN,0.013311,-0.002712,-0.001641,0.018995,0.014015,0.019636,...,0.005988,-0.001495,-0.002006,-0.000945,0.214817,1.000000,0.215462,-0.002107,-0.014587,0.003018
AMT_REQ_CREDIT_BUREAU_WEEK,0.015000,-0.017467,0.005709,NaN,0.004906,-0.001043,0.024093,0.012926,0.026397,0.014701,...,-0.007608,-0.004512,-0.006055,-0.002853,0.021885,0.215462,1.000000,-0.012199,-0.043595,0.024866
AMT_REQ_CREDIT_BUREAU_MON,-0.004202,-0.019016,-0.016955,NaN,0.009066,-0.023803,0.024042,0.056718,0.025826,0.053417,...,-0.009728,-0.003885,-0.004528,-0.005364,0.013652,-0.002107,-0.012199,1.000000,-0.035314,-0.010767
AMT_REQ_CREDIT_BUREAU_QRT,0.016465,-0.022528,-0.001697,NaN,-0.030353,-0.022028,0.006707,0.003209,0.016595,0.004426,...,-0.006256,-0.010430,-0.013997,-0.006595,0.000241,-0.014587,-0.043595,-0.035314,1.000000,0.088951


In [13]:
sorted(list(df.corr()['TARGET']))

[-0.1507200772848682,
 -0.1309687926621415,
 -0.12954752718226564,
 -0.046110202090804436,
 -0.045992703650885466,
 -0.04599007525117904,
 -0.04520611702482206,
 -0.039761625493229244,
 -0.03331296880350878,
 -0.03310334034748959,
 -0.03290405336807689,
 -0.031688266859563295,
 -0.029713532258080762,
 -0.028102952042565483,
 -0.028068889862141462,
 -0.027218252805351836,
 -0.027052645298535155,
 -0.02692248968131364,
 -0.026595365458070327,
 -0.026565709031903546,
 -0.02649079593614602,
 -0.025762022546263752,
 -0.02554521272133045,
 -0.0251158578648942,
 -0.024612157911699083,
 -0.0244225774045672,
 -0.024361055249211727,
 -0.02420732926810737,
 -0.02343507830870301,
 -0.023181578957477286,
 -0.022802532584821233,
 -0.02244149784412661,
 -0.022205024252245114,
 -0.02194979271485281,
 -0.021425035167508908,
 -0.021299647781074346,
 -0.02122574832477737,
 -0.02122539112563392,
 -0.019992588090237167,
 -0.018974830538604232,
 -0.018369065009061165,
 -0.0181927985309718,
 -0.0170464789617

In [14]:
df

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100083,0.0,0,0,0,0,0,103500.0,573628.5,24435.0,463500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100145,0.0,0,1,0,0,1,202500.0,260725.5,16789.5,198000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100179,0.0,0,1,0,1,0,202500.0,675000.0,53329.5,675000.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
100190,0.0,0,0,0,1,0,162000.0,263686.5,24781.5,238500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100295,1.0,0,0,0,1,1,225000.0,1019205.0,31032.0,774000.0,...,1,0,0,0,0.0,0.0,0.0,6.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456083,0.0,0,1,0,0,2,112500.0,361462.5,16051.5,274500.0,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0
456084,0.0,0,1,0,0,1,99000.0,675000.0,21906.0,675000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
456140,1.0,0,1,0,0,1,261000.0,711454.5,47673.0,643500.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0


In [15]:

from machinelearning import prediction


file=open("machinelearning.py","r",encoding="utf-8")
print(file.read())
file.close()
del file




# -*- coding: utf-8 -*-
"""
Created on Sat Oct 17 11:54:10 2020

@author: minimilien
"""

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

def score_personnalisé(prediction,realite):
    VP_FP_FN=list(filter(lambda x : (x[0]==0 and x[1]==0)==False ,zip(prediction,realite)))
    #print("Taille de la liste :",len(realite))
    #print("Taille de la liste sans les vrais négatifs:",len(VP_FP_FN))
    VP=len(list(filter(lambda x : x[0]==x[1] ,VP_FP_FN)))
    return round(VP*100/len(VP_FP_FN),2)

def prediction(dataframe,
               target='TARGET',
               models=[LinearSVC(),
                       RandomForestClassifier(n_estimators=750),
                       GradientBoostingClassifier(),
                       LogisticRegression()
                      ]
       

In [16]:
prediction(df)

création des échantillons
Apprentissage des modèles


C:\Users\minim\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

C:\Users\minim\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



La précision de LinearSVC() est : 90.24%
La précision de RandomForestClassifier(n_estimators=750) est : 94.33%
La précision de GradientBoostingClassifier() est : 94.42%
La précision de LogisticRegression() est : 65.09%
La précision perso de LinearSVC() est : 0.94%
La précision perso de RandomForestClassifier(n_estimators=750) est : 1.61%
La précision perso de GradientBoostingClassifier() est : 0.0%
La précision perso de LogisticRegression() est : 5.06%
